In [54]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import requests
from bs4 import BeautifulSoup
import time
import lxml
import re
import string

# Parsing XML Files from ProPublica

Last updated: 6/1/23 by Christopher Lum

### Methods

`scrape_website(url, attempts=0)`
- Takes a URL and returns the response object if successful. Retries three times on failure

`info_grabber(fp, url=True)`
- Takes the URL/filepath of an XML file. The `url` parameter indicates whether the given filepath is a URL or local
- Returns a dictionary of the row information for section VIII (along with a couple other columns)
- Older (prior to 2013) XML files have different tags and thus can't be read. Results in a row of all zeroes and `'Not Found'` in the `ba` column
- Of successfully scraped columns, has ~1.5% error rate for rows (0.13% of cells are inaccurate), only in the `Fundraising Events` column due to inconsistent labeling in XMLs
    - This error rate is lower than the human error rate
- In 2013, they had a different way of tagging zip codes, so all 2013 zips will be 0

`contact_grabber(fp, url=True)`
- Returns a dataframe of each contact in the 990 form, according to section VII
- Incomplete and edge cases aren't tested fully, as we didn't need the function.

`get_xmls(ein)`
- Takes the EIN of a company and returns a list of the URLs for each XML file that is linked on the ProPublica page for that given company

`grabber(eins, verbose=False, clean=False)`
- Takes a list of EINs and uses `get_xmls()` to find each XML file, then uses `info_grabber()` to scrape each XML for the relevant information.
- Returns a pandas DataFrame, doesn't automatically save as csv.
- Takes around 22 minutes on my laptop for 71 businesses
- The `clean` parameter automatically cleans the dataframe by removing empty rows

`find_errors(info)`
- Data columns are supposed to add up to the total, if it doesn't this method will find it
- Takes a dataframe of 990 rows from `grabber` and returns a tuple of incides that didn't properly grab all of the data
    - Preferably indiced on `EIN_YEAR`

#### `scrape_website`

In [55]:
def scrape_website(url, attempts=0):
    # Send GET request to the website
    response = requests.get(url)
    time.sleep(1)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        if attempts > 0:
            print('Success')
        return response
    elif attempts < 3:
        print("Error:", response.status_code, "Trying again.")
        time.sleep(1)
        return scrape_website(url, attempts + 1)
    else:
        # Handle the request error
        print("Error: ", response.status_code, 'Failed on:', url)
        return None


#### `info_grabber`

In [235]:
def info_grabber(fp, url=True, errors=False):
    if url:
        xml = scrape_website(fp)
        xml.encoding = 'UTF-8'
        xml = xml.text.strip()
    else:
        try:
            with open(fp, 'r') as file:
                xml = file.read()
        except FileNotFoundError:
            print(f"File not found: {fp}")
            return dict()
    if xml is None:
        return dict()
    
    soup = BeautifulSoup(xml, 'xml')

    row = dict()
    row['ba'] = soup.find('BusinessName')
    row['EIN'] = soup.find('EIN')
    row['Tax Year'] = soup.find('TaxPeriodBeginDt')
    row['Location (Zipcode)'] = soup.find('USAddress')
    row['Federate Campaigns'] = soup.find('FederatedCampaignsAmt')
    row['Membership Dues'] = soup.find('MembershipDuesAmt')
    row['Fundraising Events'] = soup.find('ContriRptFundraisingEventAmt')
    
    row['Related Organizations'] = soup.find('RelatedOrganizationsAmt')
    row['Government Grants'] = soup.find('GovernmentGrantsAmt')
    row['All Other Contributions'] = soup.find('AllOtherContributionsAmt')
    row['Noncash Contributions'] = soup.find('NoncashContributionsAmt')
    row['Total'] = soup.find('TotalContributionsAmt')
    # Can make default=0

    if row['Location (Zipcode)'] is not None:
        row['Location (Zipcode)'] = row['Location (Zipcode)'].find('ZIPCd')
    else:
        row['Location (Zipcode)'] = None

    row = dict(map(lambda item: (item[0], item[1].text if item[1] is not None else None), row.items()))

    if row['ba'] is not None:
        row['ba'] = row['ba'].strip()
    else:
        row['ba'] = 'Not found'
    if row['Tax Year'] is not None:
        row['Tax Year'] = pd.Timestamp(row['Tax Year']).year
    else:
        row['Tax Year'] = np.nan
    
    if errors:
        sumKeys = ['Federate Campaigns', 'Membership Dues', 'Related Organizations', 'Government Grants', 'All Other Contributions']
        sumAmt = sum(float(row[key]) for key in sumKeys if key in row and row[key] is not None)
        fundAmt_element = soup.find('FundraisingAmt')
        fundAmt = int(fundAmt_element.text) if fundAmt_element is not None else 0
        sumAmt += fundAmt

        if sumAmt is None or row['Total'] is None:
            pass
        elif int(sumAmt) == int(row['Total']):
            row['Fundraising Events'] = fundAmt
        else:
            pass

    row = dict(map(lambda item: (item[0], item[1] if item[1] is not None else 0), row.items()))

    return row


#### `contact_grabber`

Old, might not work

In [57]:
def contact_grabber(fp, url=True):
    '''
    Return a dataframe of the important names and their positions in an organization for each year

    :param fp: filepath of the xml file
    :param url: Whether the fp is a url or not. Default = True
    '''
    if url:
        xml = scrape_website(fp)
    else:
        xml = requests.get(fp)
    if xml is None:
        return dict()
    xml.encoding = 'UTF-8'
    xml = xml.text.strip()
    soup = BeautifulSoup(xml, 'xml')

    people = soup.find_all('Form990PartVIISectionAGrp')
    names = [x.find('PersonNm').text for x in people if x.find('PersonNm') is not None]
    titles = [x.find('TitleTxt').text for x in people if x.find('TitleTxt') is not None]
    try:
        organization = [soup.find('BusinessName').text.strip()] * len(names)
    except AttributeError:
        organization = ['Not found'] * len(names)
    try:
        year = [pd.Timestamp(soup.find('TaxPeriodBeginDt').text).year] * len(names)
    except AttributeError:
        year = [0] * len(names)
        
    return pd.DataFrame({'organization': organization, 'year': year, 'names': names, 'titles': titles})

#### `get_xmls`

In [58]:
def get_xmls(ein):
    '''
    Return a list of xml urls from ProRepublica based on an organization's EIN

    :param ein: Employer Identification number as String or Int
    '''
    url = f'https://projects.propublica.org/nonprofits/organizations/{ein}'
    html = scrape_website(url)
    if html is None:
        return []
    soup = BeautifulSoup(html.text)
    links = soup.find_all('a', class_='action xml')
    base_url = 'https://projects.propublica.org'
    # urls = [base_url + x.get('href') for x in links if x.text == '990']
    urls = [x for x in soup.find_all(class_='action xml') if re.search(r'990\b', x.text)]
    urls = [base_url + x.get('href') if x.name == 'a' else base_url + x.select_one('select.action.xml option[data-href]').get('data-href') for x in urls]
    return urls

#### `grabber`

In [59]:
def grabber(eins, verbose=False, clean=False, errors=False):
    if type(eins) == int:
        eins = [eins]
    data = []
    # contacts = pd.DataFrame()
    overall_index = 0
    for index, ein in enumerate(eins):
        print(str(index) + (' / ') + str(len(eins)))
        xmls = get_xmls(ein)
        for xml in xmls:
            if verbose:
                print(f'{overall_index} {xml}')
            if errors:
                data.append(info_grabber(xml,errors=True))
            else:
                data.append(info_grabber(xml))
            #contacts = pd.concat([contacts, contact_grabber(xml)])
            overall_index += 1
    print(str(len(eins)) + (' / ') + str(len(eins)))
    info = pd.DataFrame(data)
    if not clean:
        return info

    no_error = info.dropna().reset_index(drop=True)
    no_error.loc[:, 'EIN':'Total'] = no_error.loc[:, 'EIN':'Total'].astype(int)
    no_error.loc[:, 'EIN_YEAR'] = no_error['EIN'].astype(str) + '_' + no_error['Tax Year'].astype(str)
    no_error = no_error.set_index('EIN_YEAR')

    return no_error

#### `find_errors`

In [60]:
def find_errors(info):
    '''
    Takes a dataframe of 990 rows and returns a tuple of the indices where it failed to grab the data properly

    :param info: The dataframe of 990 rows
    '''
    accurate = info[['Federate Campaigns', 'Membership Dues', 'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions']].sum(axis=1) == info['Total']
    return tuple(accurate[~accurate].index)

### Using the `grabber()` method

`grabber()` takes a list of EINs. This was curated from the old "990 Manual Pull" spreadsheet by pasting and splitting the entire `EIN` column and putting it into a set.

In [8]:
all_eins = '''113723093
113723093
113723093
113723093
113723093
237279074
237279074
237279074
237279074
237279074
237334012
237334012
237334012
237334012
237334012
261712580
261712580
261712580
261712580
261712580
261712580
330006089
330006089
330006089
330006089
330215585
330215585
330215585
330215585
330215585
330317950
330317950
330317950
330317950
330433314
330433314
330433314
330433314
330433314
330492304
330492304
330492304
330492304
330492304
330496092
330496092
330496092
330496092
330623634
330623634
330623634
330623634
330623634
331029843
331029843
331029843
331029843
331146733
331146733
331146733
331146733
465055513
465055513
465055513
465055513
942358632
942358632
942358632
942358632
942358632
951644024
951644024
951644024
951644024
951869806
951869806
951869806
951869806
951869806
951869806
951874073
951874073
951874073
951874073
951874073
951874073
951944230
951944230
951944230
951944230
951944230
952039198
952039198
952039198
952039198
952157559
952157559
952157559
952157559
952157559
952648050
952648050
952648050
952648050
952648050
952648050
952653869
952653869
952653869
952653869
952653869
952833205
952833205
952833205
952833205
952833205
952850121
952850121
952850121
952850121
952850121
952880426
952880426
952880426
952880426
952880426
953244085
953244085
953244085
953244085
953244085
953248148
953248148
953248148
953248148
953248148
953302967
953302967
953302967
953302967
953302967
953302967
953315571
953315571
953315571
953315571
953315571
953649525
953649525
953649525
953649525
956379598
956379598
956379598
956379598
953497926
953497926
953497926
953497926
953497926
953497926
953837714
953837714
953837714
953837714
953837714
953140767
953140767
953140767
953140767
951648184
951648184
951648184
951648184
951648184
952693142
952693142
952693142
952693142
952693142
330217339
330217339
330217339
330217339
330217339
330217339
952794253
952794253
952794253
952794253
952794253
952794253
951945256
951945256
951945256
951945256
951945256
270865318
270865318
270865318
270865318
270865318
331146733
331146733
331146733
331146733
331146733
821946283
821946283
263405689
263405689
263405689
263405689
263405689
263405689
330602842
330602842
330602842
330602842
330602842
330902617
330902617
330902617
330902617
330902617
952457354
952457354
952457354
952457354
952457354
330008269
330008269
330008269
330008269
330008269
953031682
953031682
953031682
953031682
953031682
237161267
237161267
237161267
237161267
330497515
330497515
330497515
330497515
330497515
953750738
953750738
953750738
953750738
953750738
953368020
953368020
953368020
953368020
953368020
953368020
953798088
953798088
953798088
953798088
952111196
952111196
952111196
952111196
952949636
952949636
952949636
952949636
952949636
330618893
330618893
953782164
953782164
953782164
953782164
953782164
953782164
272917644
272917644
272917644
272917644
273390797
273390797
273390797
931008369
931008369
931008369
931008369
931008369
952213995
952213995
952213995
952213995
562613191
562613191
562613191
562613191
562613191
952422704
952422704
952422704
952422704
941676390
941676390
941676390
941676390
941676390
550806460
550806460
550806460
550806460
550806460
550806460
330122462
330122462
330122462
330122462
270447059
270447059
270447059
270447059
270447059
330210280
822363154
953138268
953950196'''

In [9]:
wrong_eins = '''952850121 562613191 952457354 330210280 952877102 330433314 204374795 330602842 465055513 263405689 263405689 330602842 330602842 330602842 330433314 951644024 330553621 952653869 330433314 941676390 953302967 941676390 941676390 941676390 465055513 330602842 952850121 330006089 952111196 330433314 562613191 952880426 953244085
'''

In [10]:
eins = list(set(all_eins.split()))
# eins = list(set(wrong_eins.split()))

<b>Warning</b>: `grabber()` takes a long time. The following cell will take about 20 minutes.

# 990 - Manual Pull Testing

In [11]:
info990 = grabber(eins, verbose=True)

0 / 71
0 https://projects.propublica.org/nonprofits/download-xml?object_id=202210809349300966
1 https://projects.propublica.org/nonprofits/download-xml?object_id=202121099349301417
2 https://projects.propublica.org/nonprofits/download-xml?object_id=202010669349301306
3 https://projects.propublica.org/nonprofits/download-xml?object_id=201842969349300634


KeyboardInterrupt: 

### Checking the missing cells

The following two cells should both be 1.0.

The first checks the conditional probability that if the name is not found in a row, the tax year wasn't found.

The second checks the inverse conditional probability. Sometimes, if the request attempt tried and failed three times, the row will be entirely empty, where the name is `''` and not `'Not Found'`

In [ ]:
# Every time the name isn't found, nothing is found
info990.loc[info990['ba'] == 'Not found']['Tax Year'].isna().mean()

In [ ]:
# And every time the tax year isn't found the name isn't found
(info990[['ba', 'Tax Year']].loc[info990['Tax Year'].isna()]['ba'] == 'Not found').mean()

This cell indicates how many XML files failed to be accessed

In [ ]:
info990['ba'].isna().sum()

If the following cell returns 0.0, that indicates that every time the tax year is missing, every single financial cell is zero. All of these rows can be dropped and no information is lost.

In [ ]:
info990.loc[info990['Tax Year'].isna(), 'Federate Campaigns':].sum(axis=1).sum()

### Cleaning the raw information

Assuming the prior three checks are all adequate, we can drop rows with any nan values.

In [ ]:
no_error = info990.dropna().reset_index(drop=True)
no_error.loc[:, 'EIN':'Total'] = no_error.loc[:, 'EIN':'Total'].astype(int)
no_error.loc[:, 'EIN_YEAR'] = no_error['EIN'].astype(str) + '_' + no_error['Tax Year'].astype(str)
no_error = no_error.set_index('EIN_YEAR')

### Export here!

<b>REMEMBER TO EXPORT</b>

`no_error` is the cleaned dataframe

`info990` is the raw dataframe with missing rows

In [ ]:
 no_error.to_csv('no_errors.csv')

In [ ]:
 info990.to_csv('full_table.csv')

The following line appends the rows to the bottom of the existing dataframe titled `'no_errors.csv'` instead.

It creates a new CSV and it is recommended to rename it back to `no_errors.csv` in the case it is exported successfully

In [ ]:
extended = pd.concat([pd.read_csv('no_errors.csv'), no_error]).reset_index(drop=True).drop_duplicates()
extended.to_csv('no_errors_appended.csv')

## Testing Zone

In [ ]:
url = 'https://projects.propublica.org/nonprofits/organizations/263405689'
html = scrape_website(url)
soup = BeautifulSoup(html.text)
links = soup.find_all('a', class_='action xml')
base_url = 'https://projects.propublica.org'
urls = [base_url + x.get('href') for x in links if x.text == '990']

In [ ]:
test = [x for x in soup.find_all(class_='action xml') if re.search(r'990\b', x.text)]
test2 = [base_url + x.get('href') if x.name == 'a' else base_url + x.select_one('select.action.xml option[data-href]').get('data-href') for x in test]

In [ ]:
info990

In [ ]:
extended = pd.concat([pd.read_csv('no_errors.csv'), no_error]).reset_index(drop=True).drop_duplicates()

In [ ]:
extended.loc[:, 'EIN':'Total'] = extended.loc[:, 'EIN':'Total'].astype(int)

In [ ]:
extended = extended.drop_duplicates()

In [ ]:
info = pd.read_csv('no_errors.csv')

In [ ]:
info = info.set_index('EIN_YEAR')
# accurate = info[['Federate Campaigns', 'Membership Dues', 'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions']].sum(axis=1) == info['Total']

In [ ]:
info

In [ ]:
# tuple(accurate[~accurate].index)

In [ ]:
info_EINerrs = find_errors(info)

In [ ]:
info_errs = info[info.index.isin(info_EINerrs)]

In [ ]:
info_errs = [int(err[:-7]) for (err) in info_EINerrs]
info_errs

In [ ]:
filtered_info_errs = grabber(info_errs, errors=True)

In [ ]:
info_no_errors = filtered_info_errs.dropna().reset_index(drop=True)
info_no_errors.loc[:, 'EIN':'Total'] = info_no_errors.loc[:, 'EIN':'Total'].astype(int)
info_no_errors.loc[:, 'EIN_YEAR'] = info_no_errors['EIN'].astype(str) + '_' + info_no_errors['Tax Year'].astype(str)
info_no_errors = info_no_errors.set_index('EIN_YEAR')

In [ ]:
info_no_errors.to_csv('info_no_errors.csv')
EINinfoerrs = pd.read_csv('info_no_errors.csv')
EINinfoerrs = EINinfoerrs.set_index('EIN_YEAR')

In [ ]:
find_errors(EINinfoerrs)

In [ ]:
EINinfoerrs.loc[['951874073_2015.0']]

# Testing EINs from Manual Pull

In [ ]:
csvfile_eins = pd.read_csv('990 Manual Pull - Cleaned EIN List.csv')
eins = list(csvfile_eins.get('EIN').dropna())
eins = [int(ein) for ein in eins]
#eins

In [ ]:
parsedEINs = grabber(eins)#, verbose=True)

In [ ]:
no_errorsEIN = parsedEINs.dropna().reset_index(drop=True)
no_errorsEIN.loc[:, 'EIN':'Total'] = no_errorsEIN.loc[:, 'EIN':'Total'].astype(int)
no_errorsEIN.loc[:, 'EIN_YEAR'] = no_errorsEIN['EIN'].astype(str) + '_' + no_errorsEIN['Tax Year'].astype(str)
no_errorsEIN = no_errorsEIN.set_index('EIN_YEAR')

In [18]:
no_errorsEIN.to_csv('no_errorsEIN.csv')

NameError: name 'no_errorsEIN' is not defined

#### Working with the data : Preparing Dataset

In [23]:
EINinfo = pd.read_csv('no_errorsEIN.csv')

In [12]:
np.unique(EINinfo.get('EIN'))

array([113723093, 237161267, 237279074, 237334012, 261712580, 263405689,
       270447059, 270865318, 272917644, 273390797, 330006089, 330008269,
       330122462, 330210280, 330215585, 330217339, 330317950, 330433314,
       330492304, 330496092, 330497515, 330602842, 330618893, 330623634,
       330902617, 331029843, 331146733, 465055513, 550806460, 562613191,
       821946283, 822363154, 931008369, 941676390, 942358632, 951644024,
       951648184, 951869806, 951874073, 951944230, 951945256, 952039198,
       952111196, 952157559, 952213995, 952422704, 952457354, 952648050,
       952653869, 952693142, 952794253, 952833205, 952850121, 952880426,
       952949636, 953031682, 953138268, 953140767, 953244085, 953248148,
       953302967, 953315571, 953368020, 953497926, 953649525, 953750738,
       953782164, 953798088, 953837714, 953950196, 956379598], dtype=int64)

In [30]:
EINinfo = EINinfo.set_index('EIN_YEAR')
EINinfo

,ba,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,
113723093_2020.0,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2020.0,92123,0,0,0,0,17841834,183535,0,18025369
113723093_2019.0,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2019.0,92123,0,0,0,0,9344677,182239,0,9526916
113723093_2018.0,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2018.0,92123,0,0,0,0,2505451,865020,0,3370471
113723093_2017.0,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2017.0,92123,0,0,0,0,2021667,738875,0,2760542
113723093_2016.0,Regional Task Force on the Homeless,113723093,2016.0,92123,0,6677,0,0,1922156,65214,0,1994047
...,...,...,...,...,...,...,...,...,...,...,...,...
953140767_2017.0,MCALISTER INSTITUTE FOR TREATMENT &\nEDUCATION...,953140767,2017.0,920201651,0,0,0,0,15375695,0,0,15375695
953140767_2016.0,MCALISTER INSTITUTE FOR TREATMENT &\nEDUCATION...,953140767,2016.0,920201651,0,0,0,0,11827666,0,0,11827666
953140767_2015.0,MCALISTER INSTITUTE FOR TREATMENT &\nEDUCATION...,953140767,2015.0,920201651,0,0,0,0,9361362,0,0,9361362


##### Grouped by Year

In [21]:
EIN_Year = EINinfo.groupby('Tax Year').sum().drop(labels = ['ba','EIN','Location (Zipcode)'], axis=1)
EIN_Year.loc[2015.0, 'Fundraising Events'] -= 265175
EIN_Year.to_csv('EIN_Year.csv')

In [31]:
EIN_errs = find_errors(EINinfo)
EIN_errs

('331146733_2017.0',
 '951874073_2015.0',
 '952039198_2020.0',
 '952039198_2013.0',
 '953302967_2013.0',
 '953497926_2016.0',
 '263405689_2015.0',
 '953368020_2015.0',
 '272917644_2017.0',
 '550806460_2022.0',
 '571214920_2016.0')

#### Find errors, then prep data to double filter

In [32]:
errs1 =EINinfo[EINinfo.index.isin(EIN_errs)]

In [33]:
EINerrs = [int(err[:-7]) for err in EIN_errs]

#### Double filter testing

In [34]:
parsedErrEINs = grabber(EINerrs, errors=True)

0 / 11
1 / 11
2 / 11
3 / 11
4 / 11
5 / 11
6 / 11
7 / 11
8 / 11
9 / 11
10 / 11
11 / 11


In [35]:
no_errors2 = parsedErrEINs.dropna().reset_index(drop=True)
no_errors2.loc[:, 'EIN':'Total'] = no_errors2.loc[:, 'EIN':'Total'].astype(int)
no_errors2.loc[:, 'EIN_YEAR'] = no_errors2['EIN'].astype(str) + '_' + no_errors2['Tax Year'].astype(str)
no_errors2 = no_errors2.set_index('EIN_YEAR')

In [36]:
no_errors2.to_csv('no_errors2.csv')

In [31]:
EINErrinfo = pd.read_csv('no_errors2.csv')
EINErrinfo = EINErrinfo.set_index('EIN_YEAR')
EINErrinfo_index = list(EINErrinfo.index)
#EINErrinfo_index
#EINinfo[EINinfo.index =='952039198_2021.0']

## EIN Dataset

In [32]:
def standardize_text(text):
    return re.sub(r'[{}\'‘’]'.format(string.punctuation), '', text).upper().strip().replace('\n',' ')

In [166]:
def replace_corrected_data_and_standardize(df1, df2, concat=False):
    to_std = ['Organization Name', 'EIN', 'Tax Year', 'Location (Zipcode)', 'Federate Campaigns', 'Membership Dues',
              'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions',
              'Noncash Contributions', 'Total']

    df1 = df1.reset_index()
    df2 = df2.reset_index()
    # Standardize selected columns in both dataframes
    df1[to_std] = df1[to_std].applymap(lambda x: standardize_text(x) if isinstance(x, str) else x)
    df2[to_std] = df2[to_std].applymap(lambda x: standardize_text(x) if isinstance(x, str) else x)

    if (concat==False):
        # Replace errors with corrected information
        for index_value, corrected_row in df2.iterrows():
            df1.loc[index_value] = corrected_row
        # Combine the corrected data with original data, dropping duplicates
        combined_df = df2.combine_first(df1).drop_duplicates()
    else:
        combined_df = pd.concat([df1,df2])
    
    
    # Calculate new column 'Philanthropic_Total'
    combined_df['Philanthropic_Total'] = combined_df['Total'] - combined_df['Government Grants']

    # Rename column
    #combined_df.rename(columns={'ba': 'Organization Name'}, inplace=True)

    # Save to CSV
    combined_df.to_csv('EIN_dataset.csv', index=False)

    return combined_df

In [180]:
EINinfo_copy = EINinfo.copy()
EINErrinfo_copy = EINErrinfo.copy()

EINinfo_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)
EINErrinfo_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

EIN_dataset_df = replace_corrected_data_and_standardize(EINinfo_copy, EINErrinfo_copy)

# Print the resulting DataFrame
EIN_dataset_df

,EIN_YEAR,Organization Name,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
0,331146733_2022.0,THINK DIGNITY,331146733,2022.0,92104,0,0,63722,0,213571,256658,19922,533951,320380
1,331146733_2021.0,THINK DIGNITY,331146733,2021.0,92104,0,0,45695,0,189346,273056,63404,508097,318751
2,331146733_2020.0,THINK DIGNITY,331146733,2020.0,92104,0,0,0,0,217312,305452,37406,522764,305452
3,331146733_2019.0,THINK DIGNITY,331146733,2019.0,92104,0,0,48628,0,129001,125440,43807,303069,174068
4,331146733_2018.0,THINK DIGNITY,331146733,2018.0,92104,0,0,33682,0,114747,324226,25426,472655,357908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,571162424_2014.0,KINGDOM CAUSES INC DBA CITY NET BEDROCK CREEK,571162424,2014.0,90809,0,0,0,10335,269904,449840,0,730079,460175
585,571162424_2013.0,KINGDOM CAUSES INC DBA CITY NET BEDROCK CREEK,571162424,2013.0,0,0,0,0,18375,106888,274500,0,399763,292875
621,331029843_2021.0,INFO LINE OF SAN DIEGO COUNTY,331029843,2021.0,92142,0,0,344110,0,0,813456,0,1157566,1157566
622,331029843_2020.0,INFO LINE OF SAN DIEGO COUNTY,331029843,2020.0,92142,179534,0,457183,0,0,402785,0,1039502,1039502


##### Standardize EIN dataset

In [203]:
EIN_dataset = EIN_dataset_df.to_csv('EIN_dataset_1.csv')
EIN_dataset_df

,EIN_YEAR,Organization Name,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
0,331146733_2022.0,THINK DIGNITY,331146733,2022.0,92104,0,0,63722,0,213571,256658,19922,533951,320380
1,331146733_2021.0,THINK DIGNITY,331146733,2021.0,92104,0,0,45695,0,189346,273056,63404,508097,318751
2,331146733_2020.0,THINK DIGNITY,331146733,2020.0,92104,0,0,0,0,217312,305452,37406,522764,305452
3,331146733_2019.0,THINK DIGNITY,331146733,2019.0,92104,0,0,48628,0,129001,125440,43807,303069,174068
4,331146733_2018.0,THINK DIGNITY,331146733,2018.0,92104,0,0,33682,0,114747,324226,25426,472655,357908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,571162424_2014.0,KINGDOM CAUSES INC DBA CITY NET BEDROCK CREEK,571162424,2014.0,90809,0,0,0,10335,269904,449840,0,730079,460175
585,571162424_2013.0,KINGDOM CAUSES INC DBA CITY NET BEDROCK CREEK,571162424,2013.0,0,0,0,0,18375,106888,274500,0,399763,292875
621,331029843_2021.0,INFO LINE OF SAN DIEGO COUNTY,331029843,2021.0,92142,0,0,344110,0,0,813456,0,1157566,1157566
622,331029843_2020.0,INFO LINE OF SAN DIEGO COUNTY,331029843,2020.0,92142,179534,0,457183,0,0,402785,0,1039502,1039502


In [9]:
np.unique(EINErrinfo.get('EIN'))

array([263405689, 272917644, 331146733, 550806460, 951874073, 952039198,
       953302967, 953368020, 953497926], dtype=int64)

In [ ]:
EINErrinfo = EINErrinfo.set_index('EIN_YEAR')

In [ ]:
EINErr_errs = find_errors(EINErrinfo)
EINErr_errs

In [ ]:
errs2 = EINErrinfo[EINErrinfo.index.isin(EINErr_errs)]

In [ ]:
[err for err in EINErr_errs if err in EIN_errs]

In [ ]:
pd.merge(errs1, errs2, how='inner')

In [3]:
einyearcsv = pd.read_csv('EIN_Year.csv')

In [4]:
einyearcsv

,Tax Year,Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
0,2013.0,14812168,5931,2926753,1992567,272916322,82768266,26026065,398097644
1,2014.0,14690906,10503,3451517,1718545,436176976,85800212,18083102,541848659
2,2015.0,13449748,760,3621489,1151289,490880029,91616746,17711150,600733501
3,2016.0,12270921,11824,3270531,2928524,511590186,104729345,23099826,634806291
4,2017.0,8149406,4925,3523143,2951688,578384972,112406307,24512047,705427271
5,2018.0,7121111,9375,5954813,2737085,662322346,118763690,23556060,796908420
6,2019.0,6651291,9365,5077165,6065219,816941379,147143289,24199956,981887708
7,2020.0,5730305,5050,5559680,11438182,997288792,168776475,30818037,1188337484
8,2021.0,4795227,0,7371420,1437925,923315284,131650133,33697316,1068569989
9,2022.0,0,0,279486,0,505011,7923835,19922,8716739


### New EINs

In [62]:
new_eins = [204374795,
311750839,
330553704,
330498557,
880279721,
330352148,
330352148,
951729734,
237332048]

In [236]:
new_ein990 = grabber(new_eins)

0 / 9
1 / 9
2 / 9
3 / 9
4 / 9
5 / 9
6 / 9
7 / 9
8 / 9
9 / 9


In [237]:
new_ein990[new_ein990.get('EIN') == 880279721, 330498557]

,ba,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total


In [239]:
new_EINinfo_df = new_ein990.dropna().reset_index(drop=True)
new_EINinfo_df.loc[:, 'EIN':'Total'] = new_EINinfo_df.loc[:, 'EIN':'Total'].astype(int)
new_EINinfo_df.loc[:, 'EIN_YEAR'] = new_EINinfo_df['EIN'].astype(str) + '_' + new_EINinfo_df['Tax Year'].astype(str)
new_EINinfo_df = new_EINinfo_df.set_index('EIN_YEAR')
np.unique(new_EINinfo_df.get('EIN'))

array([204374795, 237332048, 311750839, 330352148, 330553704, 951729734],
      dtype=object)

In [227]:
# Create copies of the DataFrames and rename columns in the copies
EIN_dataset_df_copy = EIN_dataset_df.copy()
EIN_dataset_df_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

new_EINinfo_copy = new_EINinfo_df.reset_index().copy()
new_EINinfo_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

# Call the function with the modified copies
new_concat_EINinfo_df = replace_corrected_data_and_standardize(EIN_dataset_df_copy, new_EINinfo_copy, concat=True).set_index('EIN_YEAR')\
                    .drop('index',axis=1).drop_duplicates()

# Print the resulting DataFrame
new_concat_EINinfo_df

array([204374795, 237332048, 311750839, 330352148, 330553704, 951729734],
      dtype=object)

In [190]:
new_EINinfo = new_EINinfo_df.to_csv('new_EINinfo')

In [191]:
new_EINerrs = find_errors(new_EINinfo_df)
new_EINerrs = np.unique([int(err[:-7]) for err in new_EINerrs])
newErrEINs = grabber(new_EINerrs, errors=True)
#EIN_dataset_df.loc[new_eins]#, new_EINinfo_df

0 / 3
1 / 3
2 / 3
3 / 3


In [210]:
new_ErrEINs = newErrEINs.dropna().reset_index(drop=True)
new_ErrEINs.loc[:, 'EIN':'Total'] = new_ErrEINs.loc[:, 'EIN':'Total'].astype(int)
new_ErrEINs.loc[:, 'EIN_YEAR'] = new_ErrEINs['EIN'].astype(str) + '_' + new_ErrEINs['Tax Year'].astype(str)
new_ErrEINs = new_ErrEINs.set_index('EIN_YEAR')
new_ErrEINs

,ba,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,
204374795_2021.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2021.0,92121,0,0,349596,0,6327592,87887254,58312927,94564442
204374795_2020.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2020.0,92121,0,0,251384,0,10838132,113522301,80913471,124611817
204374795_2019.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2019.0,92121,0,0,103012,0,5257407,75773240,58281891,81133659
204374795_2018.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2018.0,92121,0,0,316076,0,2593934,48722765,41159936,51632775
204374795_2017.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2017.0,92121,0,0,191553,0,2106601,41309129,35087496,43607283
204374795_2016.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2016.0,92121,0,0,154260,0,1623355,38415684,31883433,40193299
204374795_2015.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2015.0,92121,0,0,117922,0,2175671,31097166,25686164,33390759
204374795_2014.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2014.0,92121,0,0,143289,0,8567254,24065867,25032864,32776410
204374795_2013.0,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2013.0,0,0,0,131556,0,2230566,30090121,25346991,32452243


In [216]:
desired_order = ['Organization Name','EIN','Tax Year','Location (Zipcode)','Federate Campaigns','Membership Dues','Fundraising Events','Related Organizations','Government Grants','All Other Contributions','Noncash Contributions','Total', 'Philanthropic_Total']

In [242]:
# Create copies of the DataFrames and rename columns in the copies
new_ErrEINs_copy = new_ErrEINs.copy()
new_ErrEINs_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

#EIN_dataset_df_copy = EIN_dataset_df.copy()
#EIN_dataset_df_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

# Call the function with the modified copies
cor_EINinfo_df = replace_corrected_data_and_standardize(EIN_dataset_df, new_ErrEINs_copy)

# Print the resulting DataFrame
final_EINinfo_df = cor_EINinfo_df.set_index('EIN_YEAR').drop('index',axis=1).drop_duplicates()
final_EINinfo_df = final_EINinfo_df[desired_order]
final_EINinfo_df

,Organization Name,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,
204374795_2021.0,JACOBS CUSHMAN SAN DIEGO FOOD BANK,204374795,2021.0,92121,0,0,349596,0,6327592,87887254,58312927,94564442,88236850
204374795_2020.0,JACOBS CUSHMAN SAN DIEGO FOOD BANK,204374795,2020.0,92121,0,0,251384,0,10838132,113522301,80913471,124611817,113773685
204374795_2019.0,JACOBS CUSHMAN SAN DIEGO FOOD BANK,204374795,2019.0,92121,0,0,103012,0,5257407,75773240,58281891,81133659,75876252
204374795_2018.0,JACOBS CUSHMAN SAN DIEGO FOOD BANK,204374795,2018.0,92121,0,0,316076,0,2593934,48722765,41159936,51632775,49038841
204374795_2017.0,JACOBS CUSHMAN SAN DIEGO FOOD BANK,204374795,2017.0,92121,0,0,191553,0,2106601,41309129,35087496,43607283,41500682
...,...,...,...,...,...,...,...,...,...,...,...,...,...
571162424_2014.0,KINGDOM CAUSES INC DBA CITY NET BEDROCK CREEK,571162424,2014.0,90809,0,0,0,10335,269904,449840,0,730079,460175
571162424_2013.0,KINGDOM CAUSES INC DBA CITY NET BEDROCK CREEK,571162424,2013.0,0,0,0,0,18375,106888,274500,0,399763,292875
331029843_2021.0,INFO LINE OF SAN DIEGO COUNTY,331029843,2021.0,92142,0,0,344110,0,0,813456,0,1157566,1157566


In [243]:
final_EINinfo_df.to_csv('Phil_Dataset.csv')